In [5]:
import pandas as pd
import numpy as np
import sys
import os
from typing import IO, Any
import torch

In [14]:
path: str = "/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-2.txt" # '/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/t.txt'
path_csv: str = "/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-2.csv" # '/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/t.csv'

In [4]:
from os import listdir
from os.path import isfile, join
import re
from typing import List, Tuple

In [13]:
bool(re.search(r"csv", path_csv))

True

In [24]:
re.match(r".*\/(.*).txt$", "/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-1.txt").group(1)

'bc-1'

In [32]:
def get_paths(path: str) -> List[Tuple[str, str, str]]:
    files: list = [f for f in listdir(path) if isfile(join(path, f))]
    files_txt: list = list(filter(lambda e: bool(re.search(r"txt", e)), files))
    files_txt = list(map(lambda e: os.path.join(path, e), files_txt))
    file_names_cleaned: list = list(map(lambda e: re.match(r".*\/(.*).txt$", e).group(1), files_txt))
    files_csv: list = list(map(lambda e: os.path.join(path, e + '.csv'), file_names_cleaned))
    return list(zip(files_csv, files_txt, file_names_cleaned))


In [33]:
get_paths("/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc")

[('/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-1.csv',
  '/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-1.txt',
  'bc-1'),
 ('/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-2.csv',
  '/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-2.txt',
  'bc-2'),
 ('/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-4.csv',
  '/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-4.txt',
  'bc-4'),
 ('/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-3.csv',
  '/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/interim/bc/bc-3.txt',
  'bc-3')]

In [15]:
df = pd.read_csv(path_csv)
df

,Unnamed: 0,neg
0,0,11
1,1,13
2,2,14
3,3,18
4,4,18
5,5,26
6,6,40
7,7,50
8,8,54
9,9,64


In [8]:
class DataServer():
    """Class for Storing and Writing Data.
    
    This class stores values that are pushed to the internal list
    over a specific length for a rolling window. If a condition is met,
    the data from the rolling-window can be saved to an internal collection
    that is written to a file periodically (use interval parameter to define
    length).
    """
    def __init__(self, target_path: str, n: int = 5, interval: int = 1000):
        """Instantiate Class.

        :param target_path: Path to where file with data will be saved.
        :param n: Max length of internal list for rolling-window.
        :param interval: Length of interval in which the internal collection
            of rolling-windows is saved.
        """
        self.list: list = []
        self.collection: list = []
        self.target_path: str = target_path
        self.n: int = n
        self.interval: int = interval

    def push(self, elem: Any) -> None:
        """Push to Rolling-Window List.

        Append element to rolling window list and pop first element
        of list iff max-length is reached.
        :param elem: Element to be appended to rolling window list.
        """
        if elem is None:
            self.list.pop(0)
            return
        if len(self.list) >= self.n:
            self.list.pop(0)
            self.list.append(elem)
        else:
            self.list.append(elem)

    def rolling_window(self) -> str:
        """Get and Save Rolling-Window List.

        :returns: String of joined content of rolling-window list with appended
            new-line-character.
        """
        data: str = " ".join(list(map(lambda e: str(e)[:-1] if str(e)[-1] == '\n' else str(e), self.list)))+ "\n"
        self.collection.append(data)
        self._check_collection()
        return data
    
    def _check_collection(self) -> None:
        """Check and Save Collection.

        If length of the interval is reached, the content of the internal
        collection is written to file.
        """
        if len(self.collection) >= self.interval:
            f: IO = open(self.target_path, "a")
            f.writelines(line for line in self.collection)
            f.close()
            self.collection = []
            pass




In [ ]:
def make_data(path_target: str, path_data: str, path_csv: str, range_n: int) -> None:
    """Make Dataset.
    
    Iterate over dataset line-by-line and save data if line
    is mentioned in data-csv. To use rolling-window technique,
    the indices of specific data instances are increased by the
    length of the rolling window in one direction (`range_n`).
    If the condition is met, the whole rolling window is saved.
    To include further instances at the end of the file, another
    loop is used to account for the last instances.

    :param path_target: Path where final data will be stored.
    :param path_data: Path where raw data is stored.
    :param path_csv: Path to csv containing important rows and
        number of total raw-data instances in last row.
    :param range_n: Length of rolling-window in one direction
        (n_range + 1 + n_range).
    """
    df = pd.read_csv(path_csv)
    f: IO = open(path_data, 'r')
    n: int = range_n * 2 + 1
    indices: list = list(df['neg'].values[:-1].astype(int))
    next_neg: int = indices.pop(0)
    collector: DataServer = DataServer(path_target, n=n, interval=2)
    for i, text in enumerate(f):
        collector.push(text)
        if i + 1 == int(next_neg) + range_n:
            collector.rolling_window()
            if len(indices) != 0:
                next_neg = indices.pop(0)
            else:
                break

    begin: int = i + 1
    for j in range(begin, i + n):
        collector.push(None)
        if j + 1 == int(next_neg) + range_n:
            collector.rolling_window()
            if len(indices) != 0:
                next_neg = indices.pop(0)
            else:
                break
    f.close()

In [18]:
f: IO = open(path, 'r')
range_n = 1
n = range_n * 2 + 1
indices: list = list(df['neg'].values[:-1].astype(int))
next_neg: int = indices.pop(0)
collector = DataServer('test', n=n, interval=1000)
for i, text in enumerate(f):
    collector.push(text)
    if i + 1 == int(next_neg) + range_n:
        print(text)
        collector.rolling_window()
        # print("collector",  collector.rolling_window(), end="")
        if len(indices) != 0:
            print(indices)
            next_neg = indices.pop(0)
        else:
            break

begin: int = i + 1
print(begin)
for j in range(begin, i + n):
    collector.push(None)
    if j + 1 == int(next_neg) + range_n:
        collector.rolling_window()
        # print("collector",  collector.rolling_window(), end="")
        if len(indices) != 0:
            next_neg = indices.pop(0)
        else:
            break
f.close()

`` what do you mean ? ''

[13, 14, 18, 18, 26, 40, 50, 54, 64, 70, 71, 85, 85, 86, 89, 94, 98, 100, 104, 113, 148, 149, 157, 165, 171, 177]
besides his wife 's death , he has n't had it easy on the dating front . ''

[14, 18, 18, 26, 40, 50, 54, 64, 70, 71, 85, 85, 86, 89, 94, 98, 100, 104, 113, 148, 149, 157, 165, 171, 177]
`` what happened ? ''

[18, 18, 26, 40, 50, 54, 64, 70, 71, 85, 85, 86, 89, 94, 98, 100, 104, 113, 148, 149, 157, 165, 171, 177]
`` em , you might as well tell her , '' casey urged .

[18, 26, 40, 50, 54, 64, 70, 71, 85, 85, 86, 89, 94, 98, 100, 104, 113, 148, 149, 157, 165, 171, 177]
185


In [118]:
os.remove("/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/notebooks/test")

In [71]:
def get_ranges(df: pd.DataFrame) -> list:

    indices: list = list(df['neg'].values[:-1].astype(int))
    length: int = int(df['neg'].values[-1][3::])

    collector: list = []
    for i in range(2):
        collector.append(torch.Tensor(indices) - (i + 1))
    collector.append(torch.Tensor(indices))
    for _ in range(2):
        collector.append(torch.Tensor(indices) + i + 1)
    n_neg: torch.Tensor = torch.stack(collector, axis=1).int()
    n_neg[n_neg < 0] = 10e8
    n_neg[n_neg > length] = 10e8
    return n_neg.tolist()

In [110]:
get_ranges(df)

[[19, 18, 20, 22, 22],
 [23, 22, 24, 26, 26],
 [29, 28, 30, 32, 32],
 [31, 30, 32, 34, 34],
 [104, 103, 105, 107, 107],
 [107, 106, 108, 110, 110],
 [141, 140, 142, 144, 144],
 [173, 172, 174, 176, 176],
 [174, 173, 175, 177, 177],
 [181, 180, 182, 184, 184],
 [182, 181, 183, 185, 185],
 [191, 190, 192, 194, 194],
 [198, 197, 199, 201, 201],
 [223, 222, 224, 226, 226],
 [235, 234, 236, 238, 238],
 [252, 251, 253, 255, 255],
 [256, 255, 257, 259, 259],
 [264, 263, 265, 267, 267],
 [264, 263, 265, 267, 267],
 [269, 268, 270, 272, 272],
 [273, 272, 274, 276, 276],
 [291, 290, 292, 294, 294],
 [292, 291, 293, 295, 295],
 [307, 306, 308, 310, 310],
 [308, 307, 309, 311, 311],
 [314, 313, 315, 317, 317],
 [321, 320, 322, 324, 324],
 [325, 324, 326, 328, 328],
 [326, 325, 327, 329, 329],
 [338, 337, 339, 341, 341],
 [362, 361, 363, 365, 365],
 [365, 364, 366, 368, 368],
 [366, 365, 367, 369, 369],
 [370, 369, 371, 373, 373],
 [370, 369, 371, 373, 373],
 [383, 382, 384, 386, 386],
 [393, 392, 